## Importing modules

In [7]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model
import h5py
K.set_image_dim_ordering('th')

## Getting and formatting data

In [2]:
# Fix random seed for reproducibility
seed = 89236
numpy.random.seed(seed)

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshape to be [samples][pixels][width][height]
# Force pixel presition values to be 32bit
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

# Normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# Convert output to one hot representations
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

## Defining a model

In [3]:
def network_model():
	# Create model
	model = Sequential()
	model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(16, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

example_model = network_model()
print(example_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 24, 24)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 10, 10)        4624      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 5, 5)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 5, 5)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               51328     
__________

## Training and assessing the model

In [4]:
model = network_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Success rate: %.6f%%" % (scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 162s 3ms/step - loss: 0.3784 - acc: 0.8841 - val_loss: 0.0919 - val_acc: 0.9722
Epoch 2/10
60000/60000 [==============================] - 163s 3ms/step - loss: 0.0888 - acc: 0.9725 - val_loss: 0.0543 - val_acc: 0.9810
Epoch 3/10
60000/60000 [==============================] - 168s 3ms/step - loss: 0.0643 - acc: 0.9796 - val_loss: 0.0440 - val_acc: 0.9851
Epoch 4/10
60000/60000 [==============================] - 186s 3ms/step - loss: 0.0521 - acc: 0.9836 - val_loss: 0.0346 - val_acc: 0.9882
Epoch 5/10
60000/60000 [==============================] - 187s 3ms/step - loss: 0.0450 - acc: 0.9862 - val_loss: 0.0296 - val_acc: 0.9903
Epoch 6/10
60000/60000 [==============================] - 148s 2ms/step - loss: 0.0405 - acc: 0.9872 - val_loss: 0.0292 - val_acc: 0.9909
Epoch 7/10
60000/60000 [==============================] - 150s 2ms/step - loss: 0.0354 - acc: 0.9888 - val_loss: 0.0302 - 

## Save model

In [6]:
# Create HDF5 file
model = network_model()
model.save('model.h5')
print('model saved')


model saved
